In [82]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
path_db = data_root / ("econvalue_재조달원가.parquet")

path_to = Path("../results/econvalue/")
path_to.mkdir(parents=True, exist_ok=True)

# Open a DuckDB connection
con = duckdb.connect()

In [83]:
db = con.sql(f"SELECT * FROM read_parquet('{path_db.as_posix()}')")
print(db)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬───────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬─────────────────┬─────────────────────────────────┬────────────────┬──────────────┬────────────────┬────────────────┬─────────────┬──────────┬─────────────────────┬──────────┬────────────┬────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │  층_주_용도_코드_명   │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │    시군구명     │            법정동명             │ 재조달용도코드 │ 재조달용도명 │ 재조달구조코드 │  재조달구조명  │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │ 잔존연한 │ 재조달원가 │ 잔존재조달원가 │
│      varchar       │   varchar   │   varchar   │     varchar      │  varchar  │   varchar    │      varc

In [84]:
usage_code = pd.read_csv("../results/동_주_용도_코드별_합계.csv")
print(usage_code.dtypes)
display(usage_code)

주_용도_코드        object
주_용도_코드_명      object
Total Count     int64
dtype: object


,주_용도_코드,주_용도_코드_명,Total Count
0,01000,단독주택,4344979
1,04000,제2종근린생활시설,719292
2,03000,제1종근린생활시설,595408
3,02000,공동주택,578529
4,21000,동물및식물관련시설,440924
...,...,...,...
119,15100,일반숙박시설,1
120,19001,주유소,1
121,1000,NaN,1
122,07100,소매시장,1


In [85]:
sido_code = pd.read_csv("../data/processed/code_sido.csv", dtype=str)
print(sido_code.dtypes)
display(sido_code)

시도코드    object
시도명     object
폐지여부    object
dtype: object


,시도코드,시도명,폐지여부
0,11,서울특별시,존재
1,21,부산직할시,폐지
2,22,대구직할시,폐지
3,23,인천직할시,폐지
4,24,광주직할시,폐지
5,25,대전직할시,폐지
6,26,부산광역시,존재
7,27,대구광역시,존재
8,28,인천광역시,존재
9,29,광주광역시,존재


In [86]:
query = """
SELECT
    *,
    CASE
        WHEN regexp_matches(시군구_코드, '^[0-9]{5}$')
        THEN LEFT(시군구_코드, 2)
        ELSE NULL
    END AS 시도_코드,
    CASE
        WHEN LENGTH(층_주_용도_코드) = 5
        THEN LEFT(층_주_용도_코드, 2) || '000'
        ELSE NULL
    END AS 층_용도_대분류_코드,
FROM db
"""
result = con.sql(query)
query = """
SELECT
    r.*,
    sc.시도명 AS 시도_명,
    uc.주_용도_코드_명 AS 층_용도_대분류_명,
FROM result r
LEFT JOIN usage_code AS uc
    ON r.층_용도_대분류_코드 = uc.주_용도_코드
LEFT JOIN sido_code AS sc
    ON r.시도_코드 = sc.시도코드
"""
result2 = con.sql(query)
print(result2)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬───────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬───────────────────┬──────────────────────────┬────────────────┬──────────────┬────────────────┬────────────────┬─────────────┬──────────┬─────────────────────┬──────────┬────────────┬────────────────┬───────────┬─────────────────────┬────────────┬───────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │  층_주_용도_코드_명   │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │     시군구명      │         법정동명         │ 재조달용도코드 │ 재조달용도명 │ 재조달구조코드 │  재조달구조명  │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │ 잔존연한 │ 재조달원가 │ 잔존재조달원가 │ 시도_코드 │ 층_용도_대분류_코드 │  시도_명   │ 층_용도_대분류_명 │
│   

In [87]:
# 시도_코드 result2 value count
query = """
SELECT
    시도_코드,
    시도_명,
    COUNT(*) AS value_count
FROM result2
GROUP BY 시도_코드, 시도_명
ORDER BY 시도_코드;
"""
print(con.sql(query))

┌───────────┬────────────────┬─────────────┐
│ 시도_코드 │    시도_명     │ value_count │
│  varchar  │    varchar     │    int64    │
├───────────┼────────────────┼─────────────┤
│ 11        │ 서울특별시     │     2855920 │
│ 26        │ 부산광역시     │     1076878 │
│ 27        │ 대구광역시     │      851049 │
│ 28        │ 인천광역시     │      802598 │
│ 29        │ 광주광역시     │      422205 │
│ 30        │ 대전광역시     │      479800 │
│ 31        │ 울산광역시     │      391740 │
│ 36        │ 세종특별자치시 │      108532 │
│ 38        │ NULL           │           3 │
│ 41        │ 경기도         │     3959045 │
│ 43        │ 충청북도       │      890371 │
│ 44        │ 충청남도       │     1075878 │
│ 46        │ 전라남도       │     1177070 │
│ 47        │ 경상북도       │     1643729 │
│ 48        │ 경상남도       │     1687097 │
│ 50        │ 제주특별자치도 │      397208 │
│ 51        │ 강원특별자치도 │      798182 │
│ 52        │ 전북특별자치도 │      995093 │
│ 99        │ NULL           │        2569 │
│ NULL      │ NULL           │           2 │
├───────────┴

In [88]:
# 시도 코드 38
query = """
SELECT
    *
FROM result2
WHERE 시도_코드 = '38'
"""
print(con.sql(query))

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬──────────┬──────────┬────────────────┬──────────────┬────────────────┬──────────────┬─────────────┬──────────┬─────────────────────┬──────────┬────────────┬────────────────┬───────────┬─────────────────────┬─────────┬───────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │ 층_주_용도_코드_명 │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │ 시군구명 │ 법정동명 │ 재조달용도코드 │ 재조달용도명 │ 재조달구조코드 │ 재조달구조명 │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │ 잔존연한 │ 재조달원가 │ 잔존재조달원가 │ 시도_코드 │ 층_용도_대분류_코드 │ 시도_명 │ 층_용도_대분류_명 │
│      varchar       │   varchar   │   varchar   │     varchar      │ 

법정동 코드를 시군구 코드에도 잘못 입력한 경우

In [89]:
# 시도 코드 99
query = """
SELECT
    *
FROM result2
WHERE 시도_코드 = '99'
"""
print(con.sql(query))

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬──────────┬──────────┬────────────────┬──────────────────┬────────────────┬────────────────┬─────────────┬──────────┬─────────────────────┬────────────────────┬────────────┬────────────────┬───────────┬─────────────────────┬─────────┬───────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │ 층_주_용도_코드_명 │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │ 시군구명 │ 법정동명 │ 재조달용도코드 │   재조달용도명   │ 재조달구조코드 │  재조달구조명  │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │      잔존연한      │ 재조달원가 │ 잔존재조달원가 │ 시도_코드 │ 층_용도_대분류_코드 │ 시도_명 │ 층_용도_대분류_명 │
│      varchar       │   varchar   │  

시군구 코드에 임의의 값(99999)이 입력된 경우

시도_명 연계가 정상적으로 이루어진 경우만 최종 결과에 사용

In [90]:
# 층_용도_대분류 result2 value count
query = """
SELECT
    층_용도_대분류_코드,
    층_용도_대분류_명,
    COUNT(*) AS value_count
FROM result2
GROUP BY 층_용도_대분류_코드, 층_용도_대분류_명
ORDER BY 층_용도_대분류_코드;
"""
print(con.sql(query).df())

   층_용도_대분류_코드  층_용도_대분류_명  value_count
0        01000        단독주택      7323080
1        02000        공동주택      4482601
2        03000   제1종근린생활시설      1753371
3        04000   제2종근린생활시설      2074134
4        05000     문화및집회시설        41151
5        06000        종교시설       104241
6        07000        판매시설        49315
7        08000        운수시설         8273
8        09000        의료시설        48521
9        10000      교육연구시설       285288
10       11000       노유자시설       128834
11       12000        수련시설        10101
12       13000        운동시설        36934
13       14000        업무시설       369683
14       15000        숙박시설       201968
15       16000        위락시설        36448
16       17000          공장       789352
17       18000        창고시설       818836
18       19000  위험물저장및처리시설        63425
19       20000     자동차관련시설       120632
20       21000   동물및식물관련시설       691010
21       22000  분뇨.쓰레기처리시설        28225
22       23000     교정및군사시설        31156
23       24000      방송통신시설         7884


Z3000 근린생활시설은 03000 제1종근린생활시설으로,
Z5000 문화및집회시설은 05000 문화및집회시설으로,
Z6000 판매및영업시설은 07000 판매시설으로,
Z8000 교육연구및복지시설은 10000 교육연구시설으로,
Z9000 공공용시설은 14000 업무시설로 집계함.

In [91]:
query = """
SELECT
    *,
    CASE
        WHEN regexp_matches(시군구_코드, '^[0-9]{5}$')
        THEN LEFT(시군구_코드, 2)
        ELSE NULL
    END AS X시도,
    LEFT(층_주_용도_코드, 2) || '000' AS X대분류,
    CASE
        WHEN LENGTH(층_주_용도_코드) = 5
            AND 층_주_용도_코드 < '40000'
        THEN LEFT(층_주_용도_코드, 2) || '000'
        ELSE NULL
    END AS X대분류,
    CASE
        WHEN X대분류 = 'Z3000' THEN '03000' -- 제1종근린생활시설
        WHEN X대분류 = 'Z5000' THEN '05000' -- 문화및집회시설
        WHEN X대분류 = 'Z6000' THEN '07000' -- 판매시설
        WHEN X대분류 = 'Z8000' THEN '10000' -- 교육연구시설
        WHEN X대분류 = 'Z9000' THEN '14000' -- 업무시설
        WHEN X대분류 = 'Z9000' THEN '14000' -- 
        ELSE X대분류
    END AS 층_용도_대분류_코드
FROM db
"""
result3 = con.sql(query)

# inner join with usage_code and sido_code
query = """
SELECT
    r.*,
    CASE
        WHEN sc.시도명 IS NULL THEN NULL
        ELSE X시도
    END AS 시도_코드,
    sc.시도명 AS 시도_명,
    uc.주_용도_코드_명 AS 층_용도_대분류_명,
FROM result3 r
JOIN usage_code AS uc
    ON r.층_용도_대분류_코드 = uc.주_용도_코드
    AND uc.주_용도_코드_명 IS NOT NULL
JOIN sido_code AS sc
    ON r.X시도 = sc.시도코드
    AND sc.시도명 IS NOT NULL
"""
result4 = con.sql(query)
print(result4)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬───────────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬───────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬───────────────────┬─────────────────────────────┬────────────────┬──────────────┬────────────────┬────────────────┬─────────────┬──────────┬─────────────────────┬────────────────────┬────────────┬────────────────┬─────────┬─────────┬───────────┬─────────────────────┬───────────┬────────────┬───────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │   지역_코드_명    │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │  층_주_용도_코드_명   │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │     시군구명      │          법정동명           │ 재조달용도코드 │ 재조달용도명 │ 재조달구조코드 │  재조달구조명  │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │      잔존연한      │

In [92]:
result4.columns

['관리_건축물대장_PK',
 '시군구_코드',
 '법정동_코드',
 'long_법정동_코드',
 '지역_코드',
 '지역_코드_명',
 '주_부속_구분_코드',
 '주_부속_구분_코드_명',
 '구조_코드',
 '주_용도_코드',
 '지붕_코드',
 '층_구조_코드',
 '층_주_용도_코드',
 '층_주_용도_코드_명',
 '층_주_부속_구분_코드',
 '층_면적_제외_여부',
 '층_면적',
 '사용승인_일',
 '준공년도',
 '경과년수2024',
 '시군구명',
 '법정동명',
 '재조달용도코드',
 '재조달용도명',
 '재조달구조코드',
 '재조달구조명',
 '재조달_단가',
 '내구연한',
 '잔존비율_최소값보정',
 '잔존연한',
 '재조달원가',
 '잔존재조달원가',
 'X시도',
 'X대분류',
 'X대분류_1',
 '층_용도_대분류_코드',
 '시도_코드',
 '시도_명',
 '층_용도_대분류_명']

In [93]:
# 시도_코드 result4 value count
query = """
SELECT
    시도_코드,
    시도_명,
    COUNT(*) AS value_count
FROM result4
GROUP BY 시도_코드, 시도_명
ORDER BY 시도_코드;
"""
print(con.sql(query))

┌───────────┬────────────────┬─────────────┐
│ 시도_코드 │    시도_명     │ value_count │
│  varchar  │    varchar     │    int64    │
├───────────┼────────────────┼─────────────┤
│ 11        │ 서울특별시     │     2846595 │
│ 26        │ 부산광역시     │     1074176 │
│ 27        │ 대구광역시     │      848007 │
│ 28        │ 인천광역시     │      800498 │
│ 29        │ 광주광역시     │      421122 │
│ 30        │ 대전광역시     │      477840 │
│ 31        │ 울산광역시     │      391426 │
│ 36        │ 세종특별자치시 │      108394 │
│ 41        │ 경기도         │     3949583 │
│ 43        │ 충청북도       │      889429 │
│ 44        │ 충청남도       │     1073610 │
│ 46        │ 전라남도       │     1175901 │
│ 47        │ 경상북도       │     1639441 │
│ 48        │ 경상남도       │     1681355 │
│ 50        │ 제주특별자치도 │      396156 │
│ 51        │ 강원특별자치도 │      794290 │
│ 52        │ 전북특별자치도 │      992864 │
├───────────┴────────────────┴─────────────┤
│ 17 rows                        3 columns │
└──────────────────────────────────────────┘



In [94]:
# 층_용도_대분류 result4 value count
query = """
SELECT
    층_용도_대분류_코드,
    층_용도_대분류_명,
    COUNT(*) AS value_count
FROM result4
GROUP BY 층_용도_대분류_코드, 층_용도_대분류_명
ORDER BY 층_용도_대분류_코드;
"""
print(con.sql(query).df())

   층_용도_대분류_코드  층_용도_대분류_명  value_count
0        01000        단독주택      7322084
1        02000        공동주택      4482531
2        03000   제1종근린생활시설      1752891
3        04000   제2종근린생활시설      2073789
4        05000     문화및집회시설        41133
5        06000        종교시설       104220
6        07000        판매시설        49291
7        08000        운수시설         8273
8        09000        의료시설        48485
9        10000      교육연구시설       285263
10       11000       노유자시설       128832
11       12000        수련시설        10101
12       13000        운동시설        36925
13       14000        업무시설       369458
14       15000        숙박시설       201885
15       16000        위락시설        36412
16       17000          공장       789334
17       18000        창고시설       818772
18       19000  위험물저장및처리시설        63420
19       20000     자동차관련시설       120590
20       21000   동물및식물관련시설       691010
21       22000  분뇨.쓰레기처리시설        28224
22       23000     교정및군사시설        31156
23       24000      방송통신시설         7875


In [98]:
count = con.execute(
    "SELECT COUNT(DISTINCT 관리_건축물대장_PK) FROM result4"
).fetchone()[0]  # type: ignore
print(f"{count:,.0f}")
count = con.execute("SELECT COUNT(*) FROM result4").fetchone()[0]  # type: ignore
print(f"{count:,.0f}")
sum_area = con.execute("SELECT sum(층_면적) FROM result4").fetchone()[0]  # type: ignore
print(f"{sum_area:,.0f}")
sum_재조달원가 = con.execute("SELECT sum(재조달원가) FROM result4").fetchone()[0]  # type: ignore
print(f"{sum_재조달원가:,.0f}")
sum_잔존재조달원가 = con.execute("SELECT sum(잔존재조달원가) FROM result4").fetchone()[  # type: ignore
    0
]  # type: ignore
print(f"{sum_잔존재조달원가:,.0f}")

7,290,246
19,560,687
4,356,518,178
6,466,205,512,623,296
3,893,514,963,436,596


7,290,246 동 (vs 건축물 현황 승인통계 7,421,603 동)

19,560,687개 층

4,356,518,178 ㎡

6,466,205,512,623,296 원 (내구연한 미반영)

3,893,514,963,436,596 원 (잔존 재조달원가)

## 결과표 생성

### 건축물 규모

In [117]:
# 시도별 동수, 연면적
query = """
SELECT *
FROM (
    SELECT
        시도_코드,
        시도_명,
        COUNT(DISTINCT 관리_건축물대장_PK) AS 동수,
        SUM(층_면적) AS 연면적
    FROM result4
    GROUP BY 시도_코드, 시도_명

    UNION ALL

    SELECT
        '00' AS 시도_코드,
        '전국' AS 시도_명,
        COUNT(DISTINCT 관리_건축물대장_PK) AS 동수,
        SUM(층_면적) AS 연면적
    FROM result4
)
ORDER BY
    시도_코드;
"""
df = con.sql(query).df()
display(df)
df.to_csv(path_to / "동수_연면적_시도별.csv", index=False, encoding="utf-8-sig")

,시도_코드,시도_명,동수,연면적
0,00,전국,7290246,4.356518e+09
1,11,서울특별시,557144,6.071478e+08
2,26,부산광역시,319801,2.603761e+08
3,27,대구광역시,263763,1.906385e+08
4,28,인천광역시,211869,2.390183e+08
5,29,광주광역시,128543,1.121292e+08
6,30,대전광역시,129699,1.158731e+08
7,31,울산광역시,133725,9.672484e+07
8,36,세종특별자치시,37249,3.731917e+07
9,41,경기도,1233328,1.159354e+09


In [104]:
# 용도별 동수, 연면적
query = """
SELECT
    층_용도_대분류_코드,
    층_용도_대분류_명,
    COUNT(DISTINCT 관리_건축물대장_PK) AS 동수,
    SUM(층_면적) AS 연면적
FROM result4
GROUP BY 층_용도_대분류_코드, 층_용도_대분류_명
ORDER BY 층_용도_대분류_코드;
"""
df = con.sql(query).df()
display(df)
df.to_csv(path_to / "동수_연면적_용도별.csv", index=False, encoding="utf-8-sig")

,층_용도_대분류_코드,층_용도_대분류_명,동수,연면적
0,01000,단독주택,4142879,4.870385e+08
1,02000,공동주택,594167,1.818622e+09
2,03000,제1종근린생활시설,1090233,2.406749e+08
3,04000,제2종근린생활시설,1154957,2.940889e+08
4,05000,문화및집회시설,19763,2.428538e+07
5,06000,종교시설,41488,2.932719e+07
6,07000,판매시설,18432,6.332387e+07
7,08000,운수시설,3877,8.999707e+06
8,09000,의료시설,9067,3.276621e+07
9,10000,교육연구시설,90329,1.971683e+08


### 잔존연한 산출

In [116]:
# 시도별 잔존연한
query = """
SELECT *
FROM (
    SELECT
        시도_코드,
        시도_명,
        AVG(경과년수2024) AS 평균_경과년수,
        AVG(내구연한) AS 평균_내구연한,
        AVG(잔존연한) AS 평균_잔존연한,
        AVG(잔존비율_최소값보정) AS 평균_잔존비율
    FROM result4
    GROUP BY 시도_코드, 시도_명

    UNION ALL

    SELECT
        '00' AS 시도_코드,
        '전국' AS 시도_명,
        AVG(경과년수2024),
        AVG(내구연한),
        AVG(잔존연한),
        AVG(잔존비율_최소값보정)
    FROM result4
)
ORDER BY
    시도_코드;
"""
df = con.sql(query).df()
display(df)
df.to_csv(path_to / "잔존연한_시도별.csv", index=False, encoding="utf-8-sig")

,시도_코드,시도_명,평균_경과년수,평균_내구연한,평균_잔존연한,평균_잔존비율
0,00,전국,29.735876,45.495871,22.204107,0.476431
1,11,서울특별시,26.936572,48.601638,23.132429,0.466697
2,26,부산광역시,28.806568,46.854103,21.186745,0.437652
3,27,대구광역시,29.713907,46.449015,20.825773,0.435260
4,28,인천광역시,23.932213,47.566512,25.148186,0.520024
5,29,광주광역시,25.975781,47.104491,23.268671,0.484485
6,30,대전광역시,29.328020,47.566131,21.855094,0.449401
7,31,울산광역시,24.493782,46.322817,23.791926,0.503976
8,36,세종특별자치시,23.725169,46.068279,29.900398,0.628588
9,41,경기도,21.202187,46.264234,26.855348,0.571421


In [118]:
# 용도별 잔존연한
query = """
SELECT
    층_용도_대분류_코드,
    층_용도_대분류_명,
    AVG(경과년수2024) AS 평균_경과년수,
    AVG(내구연한) AS 평균_내구연한,
    AVG(잔존연한) AS 평균_잔존연한,
    AVG(잔존비율_최소값보정) AS 평균_잔존비율
FROM result4
GROUP BY 층_용도_대분류_코드, 층_용도_대분류_명
ORDER BY 층_용도_대분류_코드;
"""
df = con.sql(query).df()
display(df)
df.to_csv(path_to / "잔존연한_용도별.csv", index=False, encoding="utf-8-sig")

,층_용도_대분류_코드,층_용도_대분류_명,평균_경과년수,평균_내구연한,평균_잔존연한,평균_잔존비율
0,01000,단독주택,40.130697,44.578679,17.017260,0.370126
1,02000,공동주택,18.176208,49.556127,31.488625,0.632573
2,03000,제1종근린생활시설,29.238817,47.459247,21.525137,0.452415
3,04000,제2종근린생활시설,23.840763,46.184647,24.101791,0.523547
4,05000,문화및집회시설,21.600734,48.503756,28.596474,0.580945
5,06000,종교시설,29.424966,49.457830,24.789666,0.487835
6,07000,판매시설,26.367248,48.937636,25.164371,0.512520
7,08000,운수시설,20.465974,49.179258,30.711531,0.615458
8,09000,의료시설,22.726720,48.106012,26.415417,0.543460
9,10000,교육연구시설,26.487715,51.738852,27.772446,0.531338


### 재조달원가

In [122]:
# 시도별 재조달원가
query = """
SELECT *
FROM (
    SELECT
        시도_코드,
        시도_명,
        SUM(재조달원가) AS 재조달원가,
        SUM(잔존재조달원가) AS 잔존재조달원가,
    FROM result4
    GROUP BY 시도_코드, 시도_명

    UNION ALL

    SELECT
        '00' AS 시도_코드,
        '전국' AS 시도_명,
        SUM(재조달원가) AS 재조달원가,
        SUM(잔존재조달원가) AS 잔존재조달원가,
    FROM result4
)
ORDER BY
    시도_코드;
"""
df = con.sql(query).df()
display(df)
df.to_csv(path_to / "재조달원가_시도별.csv", index=False, encoding="utf-8-sig")

,시도_코드,시도_명,재조달원가,잔존재조달원가
0,00,전국,6.466206e+15,3.893515e+15
1,11,서울특별시,9.577061e+14,5.293847e+14
2,26,부산광역시,3.965733e+14,2.255369e+14
3,27,대구광역시,2.941517e+14,1.719395e+14
4,28,인천광역시,3.659069e+14,2.334805e+14
5,29,광주광역시,1.759270e+14,1.050495e+14
6,30,대전광역시,1.808207e+14,1.023528e+14
7,31,울산광역시,1.432658e+14,8.304564e+13
8,36,세종특별자치시,5.695433e+13,4.458651e+13
9,41,경기도,1.725973e+15,1.123725e+15


In [123]:
# 용도별 재조달원가
query = """
SELECT
    층_용도_대분류_코드,
    층_용도_대분류_명,
    SUM(재조달원가) AS 재조달원가,
    SUM(잔존재조달원가) AS 잔존재조달원가,
FROM result4
GROUP BY 층_용도_대분류_코드, 층_용도_대분류_명
ORDER BY 층_용도_대분류_코드;
"""
df = con.sql(query).df()
display(df)
df.to_csv(path_to / "재조달원가_용도별.csv", index=False, encoding="utf-8-sig")

,층_용도_대분류_코드,층_용도_대분류_명,재조달원가,잔존재조달원가
0,01000,단독주택,7.715069e+14,3.643520e+14
1,02000,공동주택,3.070614e+15,1.962285e+15
2,03000,제1종근린생활시설,3.395110e+14,1.863602e+14
3,04000,제2종근린생활시설,4.014103e+14,2.335391e+14
4,05000,문화및집회시설,3.894134e+13,2.471370e+13
5,06000,종교시설,5.317339e+13,3.052647e+13
6,07000,판매시설,9.544051e+13,6.057311e+13
7,08000,운수시설,1.360633e+13,9.362518e+12
8,09000,의료시설,5.617769e+13,3.356521e+13
9,10000,교육연구시설,2.894134e+14,1.732674e+14
